In [1]:
from concurrent.futures import ThreadPoolExecutor
import psycopg2
import time
from random import randint

In [2]:
DB_PARAMS = "dbname=lab-1 user=postgres host=localhost password=postgres"

In [3]:
# conn = psycopg2.connect(DB_PARAMS)
# cur = conn.cursor()

# for _ in range(100_000):
#     cur.execute("INSERT INTO user_counter(counter, version) VALUES(0, 0)")
    
# conn.commit()
# cur.close()
# conn.close()

In [4]:
conn = psycopg2.connect(DB_PARAMS)
cur = conn.cursor()

cur.execute("SELECT COUNT(*) FROM user_counter")
values_count = cur.fetchone()[0]

cur.close()
conn.close()

print(f"Кількість записів в базі даних = {values_count}")

Кількість записів в базі даних = 100000


In [5]:
def setup_db():
    conn = psycopg2.connect(DB_PARAMS)
    cursor = conn.cursor()

    cursor.execute("UPDATE user_counter SET counter = 0, version = 0" )
    conn.commit()
    
    cursor.close()
    conn.close()

In [6]:
def print_time_counter(update_counter, id):
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(update_counter, [id for _ in range(10)])

    print("--- %s seconds ---" % (time.time() - start_time))

    conn = psycopg2.connect(DB_PARAMS)
    cur = conn.cursor()
    cur.execute("SELECT counter FROM user_counter WHERE user_id = %s;", (id,))
    counter = cur.fetchone()[0]
    print(f"Кінцеве значення каунтера = {counter}, для user_id = {id}")

## Lost-update

In [7]:
def lost_update(id):
    
    conn = psycopg2.connect(DB_PARAMS)
    cur = conn.cursor()
    
    for _ in range(10_000):
        cur.execute("SELECT counter FROM user_counter WHERE user_id = %s", (id,))
        counter = cur.fetchone()[0] 
        counter += 1  
        
        cur.execute("UPDATE user_counter SET counter = %s WHERE user_id = %s", (counter, id))
        conn.commit() 

    cur.close()
    conn.close()

In [8]:
setup_db()
print_time_counter(lost_update, 1)

--- 34.10344958305359 seconds ---
Кінцеве значення каунтера = 10855, для user_id = 1


In [9]:
setup_db()
print_time_counter(lost_update, randint(1, 100_000))

--- 35.10290336608887 seconds ---
Кінцеве значення каунтера = 10962, для user_id = 57182


## In-place update


In [10]:
def in_place_update(id):
   
    conn = psycopg2.connect(DB_PARAMS)
    cur = conn.cursor()
    
    for _ in range(10_000):        
        cur.execute("UPDATE user_counter SET counter = counter + 1 WHERE user_id = %s", (id,))
        conn.commit() 

    cur.close()
    conn.close()

In [11]:
setup_db()
print_time_counter(in_place_update, 1)

--- 28.29792833328247 seconds ---
Кінцеве значення каунтера = 100000, для user_id = 1


In [12]:
setup_db()
print_time_counter(in_place_update, randint(1, 100_000))

--- 29.10147476196289 seconds ---
Кінцеве значення каунтера = 100000, для user_id = 62542


## Row-level locking

In [20]:


def row_level_locking(id):
    
    conn = psycopg2.connect(DB_PARAMS)
    cur = conn.cursor()
    
    for _ in range(10_000):
        cur.execute("SELECT counter FROM user_counter WHERE user_id = %s FOR UPDATE", (id,))
        counter = cur.fetchone()[0] + 1
      
        cur.execute("UPDATE user_counter SET counter = %s WHERE user_id = %s", (counter, id))

        conn.commit() 

    cur.close()
    conn.close()

In [24]:
setup_db()
print_time_counter(row_level_locking, 1)

--- 44.78932547569275 seconds ---
Кінцеве значення каунтера = 100000, для user_id = 1


In [25]:
setup_db()
print_time_counter(row_level_locking, randint(1, 100_000))

--- 46.249706745147705 seconds ---
Кінцеве значення каунтера = 100000, для user_id = 48279


## Optimistic concurrency control

In [16]:

def optimistic_concurrency_control(id):
   
    conn = psycopg2.connect(DB_PARAMS)
    cur = conn.cursor()
    
    for _ in range(10_000):
        while True:
            cur.execute("SELECT counter, version FROM user_counter WHERE user_id = %s", (id,))
            counter, version =cur.fetchone()
            counter += 1
            cur.execute("update user_counter set counter = %s, version = %s where user_id = %s and version = %s", (counter, version + 1, id, version))
            conn.commit()
            count = cur.rowcount
            if count > 0:
                break
            
        
    cur.close()
    conn.close()

In [26]:
setup_db()
print_time_counter(optimistic_concurrency_control, 1)

--- 206.2631115913391 seconds ---
Кінцеве значення каунтера = 100000, для user_id = 1


In [27]:
setup_db()
print_time_counter(optimistic_concurrency_control, randint(1, 100_000))

--- 207.2033429145813 seconds ---
Кінцеве значення каунтера = 100000, для user_id = 99693
